# GA Floor Heights

Wagga Wagga Data ingestion example


## Database setup

- Deploy the docker services and migrate postgres to the latest revision of the data model.

In [ ]:
# Deploy the docker services
!docker compose up -d

# Migrate to latest revision of data model
!docker compose run --rm app alembic upgrade head

## Database ingestions

- Run scripts to ingest local data, this will take a couple of minutes.

In [ ]:
# Wagga Wagga data ingestion

# File path to root data directory
DATA_ROOT="/home/lng/dev/ga-floor-height-data-model/data/"

# Set up file paths
ADDRESS=DATA_ROOT+"national_address_points/wagga_address_points.gdb/"
BUILDINGS=DATA_ROOT+"overture/2024-10-23_australia_buildings.parquet"
DEM=DATA_ROOT+"Wagga DEM/NSW_2020_DEM_1m.vrt"
CADASTRE=DATA_ROOT+"cadastre/nsw/Wagga_Lot_EPSG7844.gdb"
ZONING=DATA_ROOT+"zoning/nsw/wagga_land_zoning.gpkg"
NEXIS=DATA_ROOT+"nexis_building_data/NSW_building.csv"
VALIDATION=DATA_ROOT+"Wagga Exposure Data/Final_Wagga.UFI/Final_Wagga_floor_heights.gpkg"
OUTPUT=DATA_ROOT+"wagga_wagga.gpkg"

# Ingest the address points
!fh-cli ingest-address-points -i "{ADDRESS}"

# Ingest the building footprints, split by cadastre, and remove small buildings
!fh-cli ingest-buildings -i "{BUILDINGS}" -d "{DEM}" --split-by-cadastre "{CADASTRE}" --remove-small --join-land-zoning "{ZONING}" --land-zoning-field "LAY_CLASS"

# Join address points to building footprints, joining the largest building on each lot to an address point
!fh-cli join-address-buildings -c "{CADASTRE}" --join-largest-building

# Ingest NEXIS method floor measures, joining the largest building on each lot to a NEXIS point
!fh-cli ingest-nexis-measures -i "{NEXIS}" -c "{CADASTRE}" --join-largest-building

# Ingest Validation method floor measures, joining the largest building on each lot to a Validation point
!fh-cli ingest-validation-measures -i "{VALIDATION}" --ffh-field "floor_height_m" -c "{CADASTRE}" --join-largest-building

# Export a geopackage of the data model
!fh-cli export-ogr-file -o "{OUTPUT}"

## Stop and clear database

- Stop the docker containers and remove the docker volume to clear the data.
- Comment the top line of cell below to run.

In [ ]:
%%script echo skipping cell  # Comment this line to run this cell

# Stop all containers, removing the postgres_data volume
!docker compose down
!docker volume rm ga-floor-height-data-model_postgres_data